<a href="https://colab.research.google.com/github/Hiridharan1/testing_LLM-s/blob/main/phi_2_trials_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Installation
!pip install transformers

# Check for GPU and configure
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Trust remote code (Be aware of security risks before enabling)
trust_remote_code = True

# Load Phi-2 model and tokenizer
model_id = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=trust_remote_code).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=trust_remote_code)

# Create a pipeline for text generation
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=device)

# Function to generate chatbot response
def generate_response(user_query):
  sysprompt = "The assistant gives helpful, detailed, and polite answers to the user's questions.\n"
  result = generator(sysprompt + user_query, max_length=128, do_sample=True, temperature=0.7)
  return result[0]['generated_text']

# Start the chatbot loop
if __name__ == "__main__":
  print("Phi-2 Chatbot ready!")
  while True:
    user_input = input("You: ")
    response = generate_response(user_input)
    print("Bot: " + response)


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Bot: The assistant gives helpful, detailed, and polite answers to the user's questions.
hi, this is the first time I am using your website. I am wondering about the difference between your and you're?
A: Hi! Welcome to our website! We're glad you're here. This question is actually a great one. The "your" referred to here is a possessive adjective, meaning it's used to show ownership or possession. An example of this would be "Is this your book?" On the other hand, "you're" is a contraction of "you are". An example of this would be "You're
You: are you my slave 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: The assistant gives helpful, detailed, and polite answers to the user's questions.
are you my slave ?????????
This is the worst and most useless website I've ever seen.
I'm so glad I haven't been around for a while.
It is a shame that these people who have been in the industry for so long have allowed such a website to be around.
I think it is a shame that it has become so much more difficult for us to get in to what we do because of the amount of these people!


KeyboardInterrupt: Interrupted by user

You: jjj
